In [ ]:
pip install --upgrade pip

In [20]:
import os
import yaml

with open("environment.yml") as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in environment_data["dependencies"]:
    if isinstance(dependency, dict):
      for lib in dependency['pip']:
        os.system(f"pip install {lib}")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.


  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.


In [ ]:
!pip install transformers datasets

In [9]:
! pip install -q kaggle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
os.environ['TOKENIZERS_PARALLELISM'] = 'true' # or 'false'

NameError: name 'os' is not defined

In [12]:
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Must sign up for the competition and verify your account with a text message before downloading the data

In [15]:
! kaggle competitions download -c 'shopee-product-matching'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 99%|██████████████████████████████████████▌| 1.67G/1.68G [00:13<00:00, 121MB/s]
100%|██████████████████████████████████████| 1.68G/1.68G [00:22<00:00, 79.0MB/s]


In [21]:
!unzip -q -o ./shopee-product-matching.zip -d ./shopee-product-matching/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import numpy as np
import pandas as pd
import matplotlib

import torch
import torchvision.transforms as transforms
from torch.utils.data.dataset import Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
from modules.datasets.ImageShopeeDataset import ImageShopeeDataset
from modules.datasets.TextShopeeDataset import TextShopeeDataset
from modules.models.ResNet18EmbeddingsShopeeNet import ResNet18EmbeddingsShopeeNet
from modules.models.BERTPreTrainedEmbeddingsShopeeNet import BERTPreTrainedEmbeddingsShopeeNet
import os

ModuleNotFoundError: No module named 'matplotlib'

In [3]:
DATA_FOLDER = './shopee-product-matching/'

IS_TEST = False

def get_dataset(root=DATA_FOLDER, is_test=False):
    name = "test.csv" if is_test else "train.csv"
    df = pd.read_csv(DATA_FOLDER + name)
    images_folder = "test_images/" if is_test else "train_images/"
    df['image'] = DATA_FOLDER + images_folder + df['image']
    return df

def add_target(df):
    grouped = df.groupby('label_group')['posting_id'].apply(list)
    target = df['label_group'].map(grouped)
    new_df = df.copy()
    new_df['target'] = target
    return new_df

df = get_dataset(is_test=IS_TEST)
df = add_target(df)
train = df[df['image'].apply(lambda x: os.path.exists(x))] 

In [4]:
# Prepare DataLoaders
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text_dataset = TextShopeeDataset(train['title'].values, tokenizer)

def my_collate_fn(data):
    # TODO: Implement your this
    return tuple(data)

text_loader = torch.utils.data.DataLoader(
    text_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=my_collate_fn
)

In [5]:
from transformers import AutoTokenizer, BertForPreTraining, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForPreTraining.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Pass a list of text sequences to the tokenizer
inputs = tokenizer(["Hello, my dog is cute", "Hi, my friend's cat is adorable", "Hi, my friend's cat is adorable Hi, my friend's cat is adorable"], padding=True, return_tensors="pt")

# inputs['output_hidden_states'] = True

# Pass the inputs to the model
outputs = model(**inputs)

# Extract the last hidden states
last_hidden_states = outputs.last_hidden_state

print(last_hidden_states.shape) # (N, K, H) K is sequence length and H is hidden state 

# Compute the mean of the last hidden states for each input sequence
embeddings = last_hidden_states.mean(dim=1)

print(embeddings.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([3, 20, 768])
torch.Size([3, 768])


In [6]:
last_hidden_states

tensor([[[-0.1144,  0.1937,  0.1250,  ..., -0.3827,  0.2107,  0.5407],
         [ 0.5308,  0.3207,  0.3665,  ..., -0.0036,  0.7579,  0.0388],
         [-0.4877,  0.8849,  0.4256,  ..., -0.6976,  0.4458,  0.1231],
         ...,
         [-0.4370, -0.0448,  0.3319,  ...,  0.2621, -0.0604, -0.1064],
         [-0.3728, -0.0480,  0.4407,  ...,  0.0035, -0.1797,  0.3346],
         [-0.2671,  0.1269,  0.6542,  ...,  0.0122, -0.1564,  0.3124]],

        [[-0.0534,  0.1799,  0.0135,  ..., -0.3239,  0.2102,  0.6410],
         [ 0.5771,  0.1648,  0.0964,  ..., -0.1105,  1.0164,  0.0558],
         [-0.3150,  0.8607,  0.2674,  ..., -0.2600,  0.6345,  0.1224],
         ...,
         [ 0.0195,  0.0896,  0.5284,  ...,  0.0538,  0.0268,  0.2879],
         [-0.2774, -0.1960,  0.3721,  ...,  0.1099, -0.0820,  0.2022],
         [-0.2978, -0.2896,  0.3966,  ...,  0.2111,  0.0163,  0.1468]],

        [[-0.3906, -0.0888,  0.0866,  ..., -0.4189,  0.0276,  0.5605],
         [ 0.0110,  0.0209,  0.5217,  ...,  0

In [7]:
DEVICE = 'cuda'
bert_pretrain_model = BERTPreTrainedEmbeddingsShopeeNet()
bert_pretrain_model.to(DEVICE)

text_embeddings = []
with torch.no_grad():
    for data in tqdm(text_loader):
        # data is a tuple
        data = data[0]
        
        # Check if a GPU is available
        if torch.cuda.is_available():
            # Move the inputs to the GPU
            data = {name: tensor.to('cuda') for name, tensor in data.items()}
        else:
            print('GPU not available')
        
        embeddings = bert_pretrain_model(data)
        embeddings = embeddings.data.cpu().numpy()
        text_embeddings.extend(embeddings)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/34250 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|█████████▉| 34243/34250 [06:25<00:00, 89.88it/s]

	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 34250/34250 [06:26<00:00, 88.73it/s]


In [8]:
text_embeddings = np.stack(text_embeddings)
norms = np.linalg.norm(text_embeddings, axis=1)
text_embeddings = text_embeddings / norms[:, np.newaxis]
text_embeddings.shape

(34250, 768)

In [9]:
np.save('./text_embeddings', text_embeddings)

In [10]:
!apt install libomp-dev
!python -m pip install --upgrade faiss-gpu==1.7.2
import faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libomp-dev
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.2 MB/s eta 0:00:0000:0100:01


In [13]:
k = 100
res = faiss.StandardGpuResources()
index_text = faiss.IndexFlatIP(text_embeddings.shape[1])
# index_img = faiss.index_cpu_to_gpu(res, 0, index_img)
index_text.add(text_embeddings)
similarities_text, indexes_text = index_text.search(text_embeddings, k)

In [17]:
def calc_f1_score(targets, results):
    intersect = len(np.intersect1d(targets, results))
    return 2 * intersect / (len(targets) + len(results))


def process_for_threshold(similarities, indexes, threshold, embeddings):
    f1_score_accumulated = 0
    for i in range(len(embeddings)):
        cur_sims = similarities[i]
        cur_indexes = indexes[i]
        duplicate_indexes = cur_indexes[cur_sims >= threshold]
        results = df.iloc[duplicate_indexes]['posting_id'].values
        targets = df.iloc[i]['target']
        f1_score = calc_f1_score(targets, results)
        f1_score_accumulated += f1_score
    return f1_score_accumulated / len(embeddings)


thresholds = np.arange(0.85, 0.99, 0.01)
f1_avg_scores = []
for threshold in tqdm(thresholds):
    f1_avg = process_for_threshold(similarities_text, indexes_text, threshold, text_embeddings)
    f1_avg_scores.append(f1_avg)

100%|██████████| 15/15 [02:45<00:00, 11.00s/it]


In [18]:
import matplotlib.pyplot as plt

plt.plot(thresholds, f1_avg_scores)

# Add labels and title
plt.xlabel('Threshold for cosine similarity')
plt.ylabel('Average F1-score')
plt.title('F1-score vs threshold for cosine image similarity')
plt.grid(True)

max_f1 = max(f1_avg_scores)
max_threshold = thresholds[np.argmax(f1_avg_scores)]

# plt.annotate(f"{max_f1}", xy=(max_threshold, max_f1), xytext=(max_threshold, max_f1-0.05),
#              arrowprops=dict(facecolor='black', shrink=0.2), horizontalalignment = 'center')

# Save the plot to a file
plt.savefig('cnn-baseline-thresholds.png')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print(f"Max f1-score: {max_f1}, threshold: {max_threshold}") 